In [1]:
%env CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
%env USER=neo4j
%env PW=AlexiaAmber

env: CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
env: USER=neo4j
env: PW=AlexiaAmber


set the CONNECT variable before running the command!

In [2]:
CONNECT="$CYPHER -u $USER -p $PW"

### Before loading any data, be sure your database is empty by running this command:

!$CONNECT "MATCH (n) DETACH DELETE n"

In [5]:
!{CONNECT} "MATCH (n) DETACH DELETE n"

0 rows available after 7 ms, consumed after another 0 ms


### Third, load the airbnb data into Neo4j as follows:
    
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

In [6]:
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


In [7]:
!{CONNECT} "MATCH (n) RETURN count(n)"

+----------+
| count(n) |
+----------+
| 129444   |
+----------+

1 row available after 13 ms, consumed after another 0 ms


### Go back to your notebook and run a count for each unique node label in the graph:

In [8]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(n)"

+-----------------------------+
| labels(n)        | count(n) |
+-----------------------------+
| ["Listing"]      | 5835     |
| ["Amenity"]      | 42       |
| ["Neighborhood"] | 41       |
| ["Host"]         | 4633     |
| ["User"]         | 55917    |
| ["Review"]       | 62976    |
+-----------------------------+

6 rows available after 28 ms, consumed after another 112 ms


### Sample the data by returning the contents of 10 random nodes in one query

In [9]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 10"

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| n                                                                                                                                                                                                                           |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Listing {bedrooms: 1, cleaning_fee: 80.0, weekly_price: 700.0, listing_id: "627784", availability_365: 26, price: 120.0, accommodates: 2, name: "Charming Home in Central Austin", property_type: "House", bathrooms: 1}) |
| (:Listing {bedrooms: 2, listing_id: "528160", availability_365: 365, price: 700.0, accommodates: 7, na

### and 10 random relationships in another query.

In [10]:
!{CONNECT} "MATCH ()-[r]-> () return r LIMIT 10"

+--------------------+
| r                  |
+--------------------+
| [:IN_NEIGHBORHOOD] |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
| [:HAS]             |
+--------------------+

10 rows available after 13 ms, consumed after another 1 ms


### Q1. How many hosts are located in "Austin, Texas, United States"?

In [23]:
!{CONNECT} "MATCH (h:Host) WHERE h.location = 'Austin, Texas, United States' return count(h);"

+----------+
| count(h) |
+----------+
| 3774     |
+----------+

1 row available after 9 ms, consumed after another 5 ms


### Q2. Which listings does host_id = "4641823" have? Return the listing name, property_type, price, and availability_365 sorted by price. Limit the results to 10.

In [94]:
!{CONNECT} "MATCH (h:Host)-[:HOSTS]->(l:Listing) WHERE h.host_id = '4641823' RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 10;"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | 354                |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | 335                |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | 357                |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | 341                |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | 327                |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | 344                |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | 0                  |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | 351                |

### Q3. Which users wrote a review for listing_id = "5293632"? Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [78]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(l:Listing) WHERE l.listing_id = '5293632' RETURN u.user_id, u.name ORDER BY u.name LIMIT 10;"

+--------------------------------+
| u.user_id  | u.name            |
+--------------------------------+
| "18286390" | "Annie"           |
| "30193020" | "Carole"          |
| "16497582" | "Cory"            |
| "35022795" | "Craig And Trina" |
| "13281665" | "Dianne"          |
| "29601600" | "Hannah"          |
| "11940539" | "Jacob"           |
| "3213433"  | "Jessie"          |
| "41722221" | "Johannes"        |
| "28480571" | "Ju-Ju"           |
+--------------------------------+

10 rows available after 39 ms, consumed after another 0 ms


### Q4. Which users wrote a review for any listing which has the amenities "Washer" and "Dryer"? Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [115]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(l:Listing) WHERE (:Amenity {{name: 'Washer'}})<-[:HAS]-(l:Listing)-[:HAS]->(:Amenity {{name: 'Dryer'}}) RETURN u.user_id, u.name ORDER BY u.name LIMIT 10;"

+-------------------------------------+
| u.user_id  | u.name                 |
+-------------------------------------+
| "6524431"  | "'Ley"                 |
| "8026901"  | "(We Are) Bonnie & Ky" |
| "14689717" | "(email hidden)"       |
| "11495251" | "(email hidden)"       |
| "10251681" | "(email hidden)"       |
| "8293309"  | "(email hidden)"       |
| "15315643" | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "13381969" | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
+-------------------------------------+

10 rows available after 260 ms, consumed after another 240 ms


### Q5. Which listings have 3 bedrooms and are located in the Clarksville neighborhood? Return the listing name, property_type, price, and availability_365 sorted by price. Limit the results to 5.

In [82]:
!{CONNECT} "MATCH (l:Listing)-[]->(n:Neighborhood) WHERE l.bedrooms = 3 AND n.name = 'Clarksville' RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 5;" 

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "3BR/2.5BA Exquisite Townhouse"       | "House"         | 222.0   | 358                |
| "3BR/2.5BA Tarrytown Duplex, Austin!" | "House"         | 249.0   | 336                |
| "Austin downtown hideaway"            | "House"         | 249.0   | 364                |
| "3BD Luxury Cottage by Lake Austin"   | "House"         | 290.0   | 309                |
| "Entire Adorable Downtown House"      | "House"         | 295.0   | 309                |
+----------------------------------------------------------------------------------------+

5 rows available after 33 ms, consumed after another 1 ms


### Q6. Which amenities are the most common? Return the name of the amenity and its frequency. Sort the results by count in descending order. Limit the results to 5.

To figure out which amenity is the most common, you would need the context of its relationship with the listing. So you would have to fetch the most common amenity using listing to amenity relationship.

In [88]:
!{CONNECT} "MATCH (l:Listing)-[:HAS]->(a:Amenity) RETURN distinct a.name, count(a) ORDER BY count(a) DESC LIMIT 5"

+---------------------------------------+
| a.name                     | count(a) |
+---------------------------------------+
| "Air Conditioning"         | 5615     |
| "Wireless Internet"        | 5479     |
| "Heating"                  | 5440     |
| "Kitchen"                  | 5400     |
| "Free Parking on Premises" | 5123     |
+---------------------------------------+

5 rows available after 1 ms, consumed after another 138 ms


### Q7. Which neighborhoods have the highest number of listings? Return the neighborhood’s name and zip code (neighborhood_id) along with the number of listings they have sorted by the number of listings in descending order. Limit the results to 5.

In [60]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood) RETURN distinct n.name, count(n) ORDER BY count (n) DESC LIMIT 5;"

+-----------------------------+
| n.name           | count(n) |
+-----------------------------+
| NULL             | 3248     |
| "Hancock"        | 493      |
| "Clarksville"    | 419      |
| "East Riverside" | 414      |
| "Barton Hills"   | 182      |
+-----------------------------+

5 rows available after 1 ms, consumed after another 11 ms
